# Objectives
## Research Questions to Answer: 
### 1. What are the final rankings and scores for the season?
### 2. What is the most ideal candidate for current players?
### 3. Which statistics actually impact performance or outcome of games?
####      a. How do you value one position vs. another in fantasy?
####      b. Does the volume the yards thrown or ran count for an ideal candidate?
### 4. Potential Revenue based on contracts (and social media)


In [3]:
#Dependencies
import os
import urllib
import json
import simplejson
import csv
import pandas as pd
import pytest
import base64
import requests
from pprint import pprint
from datetime import datetime
from ohmysportsfeedspy.v1_0 import API_v1_0
from ohmysportsfeedspy.v1_1 import API_v1_1
from ohmysportsfeedspy.v1_2 import API_v1_2
from ohmysportsfeedspy import MySportsFeeds
msf = MySportsFeeds(version="1.2")

In [4]:
#Authentication
msf.authenticate("mdiaz","EC1000ltd")

# NFL Schedule Function
#### (Trying to turn this into 1 api call function)

In [13]:
#Build a url path:
#Create a dictionary of information needed/intested in
#Open Necessary Files
username = "mdiaz"
password = "EC1000ltd"
baseUrl = "https://api.mysportsfeeds.com/v1.2/pull/nfl/"
Authorization = "mdiaz" + ":" + "EC1000ltd"
format_type = [".csv", ".json", ".xml"]
nfl_teams = ["ARI","ATL","BAL","BUF","CAR","CHI","CIN","CLE",
              "DAL","DEN","DET","GB","HOU","IND","JAX","KC",
              "LAC","LAR","MIA","MIN","NE","NO","NYG","NYJ",
              "OAK","PHI","PIT","SD","SEA","SF","STL","TB","TEN","WAS"]
seasons = ["2014-2015-regular/", "2015-playoff/",
         "2015-2016-regular/", "2016-playoff/",
         "2016-2017-regular/", "2017-playoff/",
         "2017-2018-regular/", "2018-playoff/"]
#API ID Parameters:
api_params = ["?gameid=",
              "?fordate="]
#Feed Paramters that DON'T USE fordates/gameid
overall_feed = ["active_players",
        "conference_team_standings"
        "cumulative_player_stats",
        "division_team_standings",
        "full_game_schedule",
        "latest_updates",
        "overall_team_standings",
        "player_injuries",
        "playoff_team_standings"]
#Feed Parameters that use ForDates:
fordate_feed = ["current_season",
                "daily_dfs",
                "daily_game_schedule",
                "daily_player_stats",
                "roster_players",
                "scoreboard",
                "roster_players"]
#Feed Parameters that use GameID:
gameid_feed = ["game_boxscore",
               "game_playbyplay",
               "game_startinglineup"]


headers= {"Authorization": "Basic " + base64.b64encode('{}:{}'.format(username,password).encode('utf-8')).decode('ascii')}
#Building a list to append through nfl_schedule function to parse the dates, away, home team
nfl_season = []
nfl_dates = []
nfl_awayTeam = []
nfl_homeTeam = []

nfl_dateDict = pd.DataFrame({
                            "Season": nfl_season,
                            "Dates": nfl_dates,
                            "Away Team": nfl_awayTeam,
                            "Home Team": nfl_homeTeam})

nfl_dateDict

,Away Team,Dates,Home Team,Season


In [ ]:
#NFL Functions for full game, active players, standings, injuries(incomplete), latest updates
# nfl_dateDict
# 1st i get the schedule
# for each schedule/sesaon i get the date, away & home team
# the function for_date needs only the date
# the function game_id needs all 3 to run
def nfl_schedule():
    
    for season in seasons:
        #season_loop = season
        #print(season_loop)
        #season_l = season_loop   
        url2 = "https://api.mysportsfeeds.com/v1.2/pull/nfl/" + season + "full_game_schedule" + format_type[1]
        print(url2)
        nflsch = requests.get(url2, headers= headers)
#       print(nflsch.json())
        x = nflsch.json()
        games_dict = x["fullgameschedule"]["gameentry"]
        for game in games_dict:
            dateOfCurrentGame = game["date"].replace("-", "")
            #dateFormatted = dateOfCurrentGame.replace("-", "")
            awayTeamCurrentGame = game["awayTeam"]["Abbreviation"]
            homeTeamCurrentGame = game["homeTeam"]["Abbreviation"]
            print(dateOfCurrentGame  + "-" + awayTeamCurrentGame + "-" +  homeTeamCurrentGame)

            
            for types in gameid_feed:
                if season == dateOfCurrentGame:
                    try:
                        gameidURL = baseUrl + season + types + format_type[1] + api_params[0] + dateOfCurrentGame + "-" + awayTeamCurrentGame + "-" + homeTeamCurrentGame
                        print(gameidURL)
                        print('Response HTTP Status Code: {status_code}'.format(status_code=response.status_code))
                        print('Response HTTP Response Body: {content}'.format(content=response.content))
                    except requests.exceptions.RequestException:
                        print('HTTP Request failed')
                        pprint(send_request())
                elif season != dateOfCurrentGame:
                    try:
                        season_loop2 = "2014-regular/"
                        gameidURL = baseUrl + season_loop2 + types + format_type[1] + api_params[0] + dateOfCurrentGame + "-" + awayTeamCurrentGame + "-" + homeTeamCurrentGame
                        print(gameidURL)
                    except "HTTP ERROR 400":
                        pass
                    else:
                        pass


                    nfl_boxscores = requests.get(gameidURL, headers=headers)
        #                   print(nfl_boxscores.json())
                    y = nfl_boxscores.json()
                    print(json.dumps(y, indent=4, sort_keys=True))
                
            for types in fordate_feed:
                fordateURL = baseUrl + season + types + format_type[1] + api_params[1] + dateOfCurrentGame
                print(fordateURL)
                nfl_boxscores = requests.get(fordateURL, headers=headers)
#                 print(nfl_boxscores.json())
                y = nfl_boxscores.json()
                print(json.dumps(y, indent=4, sort_keys=True))

In [ ]:
#Trying to print game box scores within schedule
nfl_schedule()

## 1. What are the final rankings and scores for the season?